# Importation des modules

In [338]:
# Pour manipuler et visualiser les données
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Pour séparer et évaluer les données
from sklearn.model_selection import train_test_split, cross_validate, KFold, learning_curve, GridSearchCV, RandomizedSearchCV

# Pour préparer les données
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

# Pour créer des arbres de classification
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier

# Pour utiliser les métriques
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

# Pour exporter notre modèle
import pickle

# Création du dataset et Dummy Classifier

On commence par créer un dataset avec les variables :
<p style='color: #FFA07A'> NAICS, NoEmp, NewExist, FranchiseCode,  UrbanRural, CreateJob, RetainedJob, MIS_Status </p>

In [339]:
df = pd.read_csv("archive/SBAnational_clean.csv")

/tmp/ipykernel_27561/3668100114.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("archive/SBAnational_clean.csv")


In [340]:
# Dataframe contenant les variables explicatives
X = df[['NAICS', 'NoEmp',  'RetainedJob', 'UrbanRural','Term']]

# On transforme les valeurs de MIS_Status en 0 ou 1
y = df['MIS_Status'].astype('category').cat.codes

On crée maintenant notre jeu de données d'entraînement et de test :

In [341]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size=0.2, random_state=42, stratify=y)

On sépare les variables numériques et catégorielle :

In [342]:
var_num = ['NAICS', 'NoEmp', 'RetainedJob', 'UrbanRural',"Term"]


Puis on crée un transformateur de colonne :

In [343]:
preprocessor = make_column_transformer(
    (StandardScaler(), var_num)
)

On commence notre modélisation par un Dummy Classifier qui servira de point de comparaison :

In [344]:
dummy = make_pipeline(preprocessor, DummyClassifier())

dummy.fit(X_train, y_train)

y_pred = dummy.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: ", accuracy)

precision = precision_score(y_test, y_pred)
print("Precision: ", precision)

recall = recall_score(y_test, y_pred)
print("Recall: ", recall)

f1 = f1_score(y_test, y_pred)
print("F1-score: ", f1)

Accuracy:  0.8237819517017371
Precision:  0.8237819517017371
Recall:  1.0
F1-score:  0.9033776772854687


In [345]:
from sklearn.metrics import classification_report

# Make predictions on the test set

# Generate the classification report
dummy_report = (classification_report(y_test, y_pred))
print(dummy_report)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00     31195
           1       0.82      1.00      0.90    145830

    accuracy                           0.82    177025
   macro avg       0.41      0.50      0.45    177025
weighted avg       0.68      0.82      0.74    177025



/home/apprenant/miniconda3/envs/potages/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/apprenant/miniconda3/envs/potages/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/apprenant/miniconda3/envs/potages/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_

# Random Forest Classifier

Nous allons maintenant utiliser un Random Forest Classififier sans paramètres pour le comparer avec notre Dummy Classifier :

In [346]:
# Entraînement du modèle
rf_pipe = make_pipeline(preprocessor, RandomForestClassifier())
rf_pipe.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('standardscaler',
                                                  StandardScaler(),
                                                  ['NAICS', 'NoEmp',
                                                   'RetainedJob', 'UrbanRural',
                                                   'Term'])])),
                ('randomforestclassifier', RandomForestClassifier())])

On affiche l'importance des features :

In [347]:
importances = rf_pipe[-1].feature_importances_
features = pd.DataFrame({"feature": X_train.columns, "importance": importances})
features

,feature,importance
0,NAICS,0.180534
1,NoEmp,0.060042
2,RetainedJob,0.048241
3,UrbanRural,0.033436
4,Term,0.677747


In [348]:
y_pred = rf_pipe.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: ", accuracy)

precision = precision_score(y_test, y_pred)
print("Precision: ", precision)

recall = recall_score(y_test, y_pred)
print("Recall: ", recall)

f1 = f1_score(y_test, y_pred)
print("F1-score: ", f1)

Accuracy:  0.9171190509814998
Precision:  0.9436949432348682
Recall:  0.9564561475690873
F1-score:  0.9500326939843068


In [349]:
y_pred = rf_pipe.predict(X_test)

In [350]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.78      0.73      0.76     31195
           1       0.94      0.96      0.95    145830

    accuracy                           0.92    177025
   macro avg       0.86      0.84      0.85    177025
weighted avg       0.92      0.92      0.92    177025



En utilisant un Random Forest Classifier sans paramètres, on obtient de bons résultats, proches de notre Dummy Classifier. <br>
On va essayer d'affiner notre modèle avec une cross-validation puis en cherchant les meilleurs hyperparamètres :

In [351]:
# scoring = {'recall': 'recall', 'f1': 'f1', 'precision': 'precision', 'accuracy': 'accuracy'}
# cross_val = cross_validate(rf_pipe, X_train, y_train, cv=5, scoring=scoring)

In [352]:
# # On affiche les différentes métriques avec leur écart-type
# print("Accuracy: %0.3f (+/- %0.3f)" % (cross_val['test_accuracy'].mean(), cross_val['test_accuracy'].std() * 2))
# print("Precision: %0.3f (+/- %0.3f)" % (cross_val['test_precision'].mean(), cross_val['test_precision'].std() * 2))
# print("Recall: %0.3f (+/- %0.3f)" % (cross_val['test_recall'].mean(), cross_val['test_recall'].std() * 2))
# print("F1: %0.3f (+/- %0.3f)" % (cross_val['test_f1'].mean(), cross_val['test_f1'].std() * 2))

Les résultats obtenus sont quasiment les mêmes que ceux obtenus sans cross validation. On recherche maintenant des hyperparamètres susceptibles d'améliorer le modèle :

In [353]:
""" # # On définit les paramètres à rechercher avec un intervalle de recherche
param_dist = {'n_estimators': np.arange(20, 25, 5), 
              'max_depth': np.arange(1, 11),
      'min_samples_split': np.arange(2, 10),
               'min_samples_leaf': np.arange(1, 10),
               'criterion': ['gini']}

 # On crée un objet Random Search 
random_search = RandomizedSearchCV(rf_pipe[-1], param_distributions=param_dist,
                                    n_iter=50, cv=5, n_jobs=3)

 # On entrâine le modèle
random_search.fit(X_train, y_train)  """

" # # On définit les paramètres à rechercher avec un intervalle de recherche\nparam_dist = {'n_estimators': np.arange(20, 25, 5), \n              'max_depth': np.arange(1, 11),\n      'min_samples_split': np.arange(2, 10),\n               'min_samples_leaf': np.arange(1, 10),\n               'criterion': ['gini']}\n\n # On crée un objet Random Search \nrandom_search = RandomizedSearchCV(rf_pipe[-1], param_distributions=param_dist,\n                                    n_iter=50, cv=5, n_jobs=3)\n\n # On entrâine le modèle\nrandom_search.fit(X_train, y_train)  "

In [354]:
 # On récupère les meilleurs hyperparamètres trouvés
best_params = random_search.best_params_

In [355]:
print(best_params)

{'n_estimators': 95, 'min_samples_split': 1, 'min_samples_leaf': 2, 'criterion': 'gini'}


'columntransformer__standardscaler__with_std': True,
 'randomforestclassifier__bootstrap': True,
  'randomforestclassifier__ccp_alpha': 0.0,
   'randomforestclassifier__class_weight': None,
    'randomforestclassifier__criterion': 'gini', 
    'randomforestclassifier__max_depth': None,
     'randomforestclassifier__max_features': 'sqrt', max_leaf_nodes': None,
      'randomforestclassifier__max_samples': None,
       'randomforestclassifier__min_impurity_decrease': 0.0,
        'randomforestclassifier__min_samples_leaf': 1,
         'randomforestclassifier__min_samples_split': 2,
          'randomforestclassifier__min_weight_fraction_leaf': 0.0,
           'randomforestclassifier__n_estimators': 100, 'randomforestclassifier__n_jobs': None,
            'randomforestclassifier__oob_score': False, 'randomforestclassifier__random_state': None,
             'randomforestclassifier__verbose': 0, 'randomforestclassifier__warm_start': False}

In [356]:
# Entraînement du modèle
rf_pipe = make_pipeline(preprocessor, RandomForestClassifier(**{
    'n_estimators': 100,
     'min_samples_split': 2,
      'min_samples_leaf': 1,
       'max_depth': None,
        'criterion': 'gini' }))
rf_pipe.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('standardscaler',
                                                  StandardScaler(),
                                                  ['NAICS', 'NoEmp',
                                                   'RetainedJob', 'UrbanRural',
                                                   'Term'])])),
                ('randomforestclassifier', RandomForestClassifier())])

In [357]:
from sklearn.metrics import classification_report

# Make predictions on the test set
y_pred = rf_pipe.predict(X_test)

# Generate the classification report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.78      0.73      0.76     31195
           1       0.94      0.96      0.95    145830

    accuracy                           0.92    177025
   macro avg       0.86      0.85      0.85    177025
weighted avg       0.92      0.92      0.92    177025



Cherchons autour de cette zone , 

In [358]:
 # On définit les paramètres à rechercher avec un intervalle de recherche
param_dist = {'n_estimators': np.arange(95, 100, 5), 
      'min_samples_split': np.arange(1,3),
               'min_samples_leaf': np.arange(1, 3),
               'criterion': ['gini']}


In [359]:

 """ # On crée un objet Random Search 
random_search = RandomizedSearchCV(rf_pipe[-1], param_distributions=param_dist,
                                    n_iter=50, cv=5, n_jobs=3)

 # On entrâine le modèle
random_search.fit(X_train, y_train)   """

' # On crée un objet Random Search \nrandom_search = RandomizedSearchCV(rf_pipe[-1], param_distributions=param_dist,\n                                   n_iter=50, cv=5, n_jobs=3)\n\n# On entrâine le modèle\nrandom_search.fit(X_train, y_train)   '

In [360]:
#best_params = random_search.best_params_

In [361]:
print(best_params)

{'n_estimators': 95, 'min_samples_split': 1, 'min_samples_leaf': 2, 'criterion': 'gini'}


In [362]:
rf_pipe = make_pipeline(preprocessor, RandomForestClassifier(**{
    'n_estimators': 95,
     'min_samples_split': 1,
      'min_samples_leaf': 2,
       'max_depth': None,
        'criterion': 'gini' }))
rf_pipe.fit(X_train, y_train) 

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('standardscaler',
                                                  StandardScaler(),
                                                  ['NAICS', 'NoEmp',
                                                   'RetainedJob', 'UrbanRural',
                                                   'Term'])])),
                ('randomforestclassifier',
                 RandomForestClassifier(min_samples_leaf=2, min_samples_split=1,
                                        n_estimators=95))])

In [363]:
y_pred = rf_pipe.predict(X_test)

In [364]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.81      0.75      0.78     31195
           1       0.95      0.96      0.96    145830

    accuracy                           0.93    177025
   macro avg       0.88      0.86      0.87    177025
weighted avg       0.92      0.93      0.92    177025

